In [2]:
# Step 1: Install tools
!apt-get update > /dev/null
!apt-get install -y nasm xorriso grub-pc-bin mtools > /dev/null

# Step 2: Write voibcore.asm
asm_code = """
[org 0x7c00]

start:
    call clear_screen

.loop:
    call wait_key
    call draw_raw
    jmp .loop

wait_key:
    mov ah, 0
    int 0x16       ; read key to AL
    ret

draw_raw:
    mov ah, 0x0E   ; teletype BIOS output
    mov bh, 0x00
    mov bl, 0x07   ; light gray on black
    int 0x10
    ret

clear_screen:
    mov ah, 0x06
    mov al, 0
    mov bh, 0x07
    mov cx, 0x0000
    mov dx, 0x184F
    int 0x10
    ret

times 510 - ($ - $$) db 0
dw 0xAA55
"""
with open("voibcore.asm", "w") as f:
    f.write(asm_code)

# Step 3: Assemble to binary
!nasm -f bin voibcore.asm -o boot.img

# Step 4: Prepare ISO structure
!mkdir -p iso/boot/grub
!cp boot.img iso/boot/boot.img

# Step 5: Write GRUB config
with open("iso/boot/grub/grub.cfg", "w") as f:
    f.write("""
set timeout=0
set default=0

menuentry "VOIBCORE" {
  linux16 /boot/boot.img
}
""")

# Step 6: Build ISO
!grub-mkrescue -o voibcore.iso iso > /dev/null 2>&1 || \
 xorriso -as mkisofs -R -b boot/boot.img -no-emul-boot -boot-load-size 4 -boot-info-table -o voibcore.iso iso

# Step 7: Download ISO
from google.colab import files
files.download("voibcore.iso")


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>